In [8]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from dotenv import load_dotenv

In [9]:
# DB 연결
load_dotenv()

conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_DATABASE")
)

cursor = conn.cursor()

In [12]:
# 쿼리 정의 (업종별 월평균 소비 EDA용)
query = """
SELECT
  r.sigungu_code AS city_code,
  ic.sub_category AS industry,
  DATE_FORMAT(ac.ymd, '%Y-%m') AS ym,
  SUM(ac.amt) AS total_amt,
  SUM(ac.cnt) AS total_cnt
FROM aggregated_consumption ac
JOIN regions r ON ac.region_id = r.id
JOIN industry_categories ic ON ac.industry_id = ic.id
GROUP BY r.sigungu_code, ic.sub_category, ym
ORDER BY r.sigungu_code, ym, total_amt DESC;
"""

df = pd.read_sql(query, conn)
conn.close()

df

/var/folders/36/pdpmp_4s2wv687klck6zglrm0000gn/T/ipykernel_75323/2379537012.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,city_code,industry,ym,total_amt,total_cnt
0,41111,종합소매점,2025-01,3.283674e+10,1760589.0
1,41111,연료판매,2025-01,1.442784e+10,306634.0
2,41111,차량관리/서비스,2025-01,1.341227e+10,278535.0
3,41111,기업,2025-01,1.058326e+10,242231.0
4,41111,한식,2025-01,8.500557e+09,278578.0
...,...,...,...,...,...
3709,41650,부동산,2025-03,1.838844e+06,123.0
3710,41650,금융상품/서비스,2025-03,1.320019e+06,109.0
3711,41650,유아용품,2025-03,1.154458e+06,20.0
3712,41650,교통서비스,2025-03,6.923070e+05,4.0
